In [2]:
# https://www.geeksforgeeks.org/creating-tabbed-widget-with-python-tkinter/#
# https://www.pythontutorial.net/tkinter/tkinter-treeview/

import tkinter as tk
from tkinter import ttk
import pandas as pd
import csv
import random

# Features we are hoping to implement
#   * Get a random review by clicking a button; DONE
#   * Sort tables by clicking on columns ; DONE
#   * Add data for amenities, etc...; DONE
#   * Make reviews scrollable; DONE
#   * Improve GUI aesthetics?
#   * Add tab for making quantitative comparisons between locations
#       * Comparisons between average apartment price, reviews, commute, etc?..; DONE
#       * May be more practical to process data in a csv, then import as table; SCRAPPED

# adding the commute table made some reviews not completely visible, so we may need to implement sub-tabs

class MainWindow:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Binghamton Apartment Comparison Tool")
        self.root.geometry('1100x800')
        self.init_tabs()
        
        self.load_info()
        self.import_info()
        
        self.compare_price()
        self.compare_rating()
        self.compare_commute()
        
        self.import_averages()
            
        self.review_frames = []    
            
        self.get_review(0)
        self.get_review(1)
        self.get_review(2)
        self.get_review(3)
        self.review_button()

    def init_tabs(self):
        self.tabs_widget = ttk.Notebook(self.root)
        self.all_tabs = []
        
        # Note that a lot of functions here depends on the order given by this tuple
        # Since it's difficult to match tabs to their name, assume self.all_tabs is a list
        # in the order of [UClubTab, HawleyTab, TwinRiverTab, PrintingHouseTab]
        locations = ("UClub", "20 Hawley", "Twin River Commons", "Printing House")
        for location in locations:
            new_tab = ttk.Frame(self.tabs_widget)
            self.tabs_widget.add(new_tab, text=location)
            self.all_tabs.append(new_tab)
        new_tab = ttk.Frame(self.tabs_widget)
        self.tabs_widget.add(new_tab, text="Compare Locations")
        self.all_tabs.append(new_tab)
        self.tabs_widget.pack()
        
    def load_info(self):
        # Loading room information
        floor_plan_files = ("UClub/uclub_apartments.csv",
                            "20 Hawley/20hawley_apartments.csv",
                            "Twin River Commons/twin_river_commons_apartments.csv",
                            "Printing House/printing_house_floor_plans.csv")
        self.floor_plans = []
        for file_name in floor_plan_files:
            # We use pandas instead of csv.reader to improve memory usage and
            # handle unexpected characters
            fp_dataframe = pd.read_csv(file_name)
            self.floor_plans.append(fp_dataframe)
    
        # Loading review information
        review_files = ("Reviews/google_reviews_uclub.csv",
                        "Reviews/google_reviews_20hawley.csv",
                        "Reviews/google_reviews_twinriver.csv",
                        "Reviews/google_reviews_printinghouse.csv")
        self.reviews = []
        for file_name in review_files:
            review_dataframe = pd.read_csv(file_name)
            self.reviews.append(review_dataframe)
            
        # Loading amenities
        amenity_files = ("UClub/uclub_amenities.csv",
                         "20 Hawley/20hawley_amenities.csv",
                         "Twin River Commons/twin_river_commons_amenities.csv",
                         "Printing House/printing_house_amenities.csv")
        self.amenities = []
        for file_name in amenity_files:
            amenity_dataframe = pd.read_csv(file_name)
            self.amenities.append(amenity_dataframe)
            
        # Loading commute information
        commute_files = ('UClub/uclub_commute.txt',
                        '20 Hawley/20hawley_commute.csv',
                        'Twin River Commons/twin_river_commons_commute.csv',
                        'Printing House/printing_house_commute.csv')
        self.commute_times = []
        for file_name in commute_files:
            commute_dataframe = pd.read_csv(file_name)
            self.commute_times.append(commute_dataframe)
        
    def import_info(self):
        # Adding the information we loaded using load_info() into tables, to be packed later
        # Use the syntax mytable.insert("", tk.END, values=(column1value, column2value, ...))
        
        # We start with the floor plans.
        self.fp_tables = []
        
        for index, floor_plan in enumerate(self.floor_plans):
            # We choose the tab we want to add the table to by the floor plan's list index
            current_tab = self.all_tabs[index]
            table_headings = floor_plan.columns.tolist()
            new_table = ttk.Treeview(current_tab, columns=table_headings, show="headings")
            for heading in table_headings:
                new_table.heading(heading, text=heading, command=\
                                  lambda table=new_table, col=heading:\
                                  self.sort_column(table, col, False))
            for index, row in floor_plan.iterrows():
                row = row.tolist()
                new_table.insert("", tk.END, values=row)
            new_table.pack(side=tk.TOP)
            self.fp_tables.append(new_table)
        
#####  may not need this now since the random review generator is built
        
#         # We do reviews next
#         self.review_tables = []

#         for index, review_data in enumerate(self.reviews):
#             # Creates a frame for our table and scroll bar
#             current_tab = self.all_tabs[index]
#             table_frame = tk.Frame(current_tab)
#             table_frame.pack()
            
#             # Creates the table and adds the relevant data
#             table_headings = review_data.columns.tolist()
#             new_table = ttk.Treeview(table_frame, columns=table_headings, show="headings", height=10)
#             for heading in table_headings:
#                 new_table.heading(heading, text=heading, command=\
#                                   lambda table=new_table, col=heading:\
#                                   self.sort_column(table, col, False))
#             for index, row in review_data.iterrows():
#                 row = row.tolist()
#                 new_table.insert("", tk.END, values=row)
#             new_table.pack(side=tk.LEFT)
#             self.review_tables.append(new_table)
            
#             # Creates and configures the vertical scroll bar
#             vertical_scroll = ttk.Scrollbar(table_frame, orient="vertical", command=new_table.yview)
#             vertical_scroll.pack(side=tk.RIGHT, fill=tk.Y)
#             new_table.configure(yscrollcommand=vertical_scroll.set)
        
        # We do amenities next
        self.amenity_tables = []
        # We use this for holding our amenity_tables and commute_tables, to group them together
        self.group_frames = []

        for index, amenity_data in enumerate(self.amenities):
            # Creates a frame for our table and scroll bar
            current_tab = self.all_tabs[index]
            group_frame = tk.Frame(current_tab)
            group_frame.pack(pady=10)
            self.group_frames.append(group_frame)
            table_frame = tk.Frame(group_frame)
            table_frame.pack(side=tk.LEFT, padx=20)
            
            # Creates the table and adds the relevant data
            table_headings = amenity_data.columns.tolist()
            new_table = ttk.Treeview(table_frame, columns=table_headings, show="headings", height=6)
            for heading in table_headings:
                new_table.heading(heading, text=heading, command=\
                                  lambda table=new_table, col=heading:\
                                  self.sort_column(table, col, False))
                new_table.column(heading, width=500)
            for index, row in amenity_data.iterrows():
                row = row.tolist()
                new_table.insert("", tk.END, values=row)
            new_table.pack(side=tk.LEFT)
            self.amenity_tables.append(new_table)
            
            # Creates and configures the vertical scroll bar
            vertical_scroll = ttk.Scrollbar(table_frame, orient="vertical", command=new_table.yview)
            vertical_scroll.pack(side=tk.RIGHT, fill=tk.Y)
            new_table.configure(yscrollcommand=vertical_scroll.set)
            
        # We do commute times next
        self.commute_tables = []
        
        for index, commute_data in enumerate(self.commute_times):
            table_frame = tk.Frame(self.group_frames[index])
            table_frame.pack(side=tk.RIGHT)
            
            table_headings = commute_data.columns.tolist()
            new_table = ttk.Treeview(table_frame, columns=table_headings, show='headings', height = 4)
            for heading in table_headings:
                new_table.heading(heading, text=heading, command=\
                                 lambda table=new_table, col=heading:\
                                 self.sort_column(table, col, False))
                new_table.column(heading)
            for index, row in commute_data.iterrows():
                row = row.tolist()
                new_table.insert('', tk.END, values=row)
            new_table.pack()
            self.commute_tables.append(new_table)

    def get_review(self, index):
        # if a review is already present, we get rid of it by destroying the frame it is contained in
        if len(self.review_frames) >= 4:
            old_frame = self.review_frames[index]
            self.review_frames.remove(old_frame)
            old_frame.destroy()
        current_tab = self.all_tabs[index]
        # we create a frame to contain the review
        review_frame = tk.Frame(current_tab)
        review_frame.pack()
        self.review_frames.insert(index, review_frame)
        current_table = self.reviews[index]
        # selects a random row from the table
        x = random.randint(0, len(current_table)-1)
        date = tk.Label(review_frame, text='Review from ' + str(current_table['date'][x]))
        date.pack()
        rating = tk.Label(review_frame, text='Rating: ' + str(current_table['rating'][x]))
        rating.pack()
        # displays the review text, wraplength limits the number of characters per line so the entire text is visible
        review = tk.Label(review_frame, text=current_table['review_text'][x], wraplength = 1000)
        review.pack()

    def review_button(self):
        tk.Button(self.all_tabs[0], text='Get a new review', command=lambda:self.get_review(0)).pack(side=tk.BOTTOM)
        tk.Button(self.all_tabs[1], text='Get a new review', command=lambda:self.get_review(1)).pack(side=tk.BOTTOM)
        tk.Button(self.all_tabs[2], text='Get a new review', command=lambda:self.get_review(2)).pack(side=tk.BOTTOM)
        tk.Button(self.all_tabs[3], text='Get a new review', command=lambda:self.get_review(3)).pack(side=tk.BOTTOM)
       
#     This used to have a function, but now we use it as an example for how to create and insert
#     data into tables.
#     def init_tables(self):
#         for tab in self.all_tabs:
#             new_table = ttk.Treeview(tab, columns=columns, show="headings")
#             for column in columns:
#                 new_table.heading(column, text=column)
#             new_table.insert("", tk.END, values=("value1", "value2", ...))
#             new_table.pack()
#             self.all_tables.append(new_table)
    
    # https://stackoverflow.com/questions/1966929/tk-treeview-column-sort
    def sort_column(self, table, column, reverse):
        # Explanation of this code:
            # We want to sort the tables by the values of a specific "column", identified by its heading
            # table.get_children("") retrieves the iids (identification numbers) of every row in the table
            # table.set(row_iid, column) retrieves the value found in the specified row and column 
            # The list comprehension returns a list of tuples (row_value, row_iid) that we proceed to sort
        
        # Explanation of how to use this table in a function:
            # To add sorting functionality to a table column, use the syntax:
            # my_table.heading(column_heading, command= lambda table=my_table, col=column_heading:\
            #                  self.sort_column(table, col, False))
            # Where: my_table is your table
            #        column_heading is the id of your column header
            # Leave everything else unchanged

        row_values = [(table.set(row_iid, column), row_iid) for row_iid in table.get_children("")]
        row_values.sort(reverse=reverse)
        
        # Moves every row to their newly-sorted index position
        for index, (value, iid) in enumerate(row_values):
            table.move(iid, '', index)
        
        # Sets the table heading command so it sorts in reverse upon next click 
        table.heading(column, command=\
                      lambda table=table, col=column:\
                      self.sort_column(table, col, not reverse))
    
    def compare_price(self):
        # taking the average of the rent/price column of each housing dataframe, not done for U Club yet
        # could be done with a loop once we standardize the csvs to be uniform
        
        # https://stackoverflow.com/questions/31521526/convert-currency-to-float-and-parentheses-indicate-negative-amounts/31521773
        u_price = self.floor_plans[0]['RENT'].replace( '[\$,)]','', regex=True).astype(int)
        hawley_price = self.floor_plans[1]['RENT'].replace( '[\$,)]','', regex=True).astype(int)
        twin_price = self.floor_plans[2]['RENT'].replace( '[\$,)]','', regex=True).astype(int)
        printing_price = self.floor_plans[3]['RENT'].replace( '[\$,)]','', regex=True).astype(int)
        
        # https://stackoverflow.com/questions/15619096/add-zeros-to-a-float-after-the-decimal-point-in-python
        u_avg = format(round(u_price.mean(), 2), '.2f')
        hawley_avg = format(round(hawley_price.mean(), 2), '.2f')
        twin_avg = format(round(twin_price.mean(), 2), '.2f')
        printing_avg = format(round(printing_price.mean(), 2), '.2f')
        self.average_prices = [u_avg, hawley_avg, twin_avg, printing_avg]
        price_frame = tk.Frame(self.all_tabs[4])
        price_frame.pack()
        
        u_club = tk.Label(price_frame, text=f'Average rent for U Club: ${u_avg}').pack()
        hawley = tk.Label(price_frame, text=f'Average rent for 20 Hawley: ${hawley_avg}').pack()
        twin = tk.Label(price_frame, text=f'Average rent for Twin River Commons: ${twin_avg}').pack()
        printing = tk.Label(price_frame, text=f'Average rent for Printing House: ${printing_avg}').pack()
        
        # displays the individual prices
        best = 2000
        for price in self.average_prices:
            if float(price) < best:
                best = float(price)
        best = format(best, '.2f')
        
        if best == u_avg:
            place = 'U Club'
        elif best == hawley_avg:
            place = '20 Hawley'
        elif best == twin_avg:
            place = 'Twin River Commons'
        elif best == printing_avg:
            place = 'Printing House'
        
        # displays the lowest average price
        text = tk.Label(price_frame, text=f'The cheapest rent on average is ${best}, at {place}.')
        text.config(font=('Sans Serif', 25, 'bold'))
        text.pack()
        
    def compare_rating(self):
        self.average_rating_labels = []
        self.average_ratings = [] # for compare_ratings()
        with open("Reviews/google_average_ratings.csv", newline="") as avg_ratings_file:
            csv_reader = list(csv.reader(avg_ratings_file))
            average_ratings = csv_reader[1]
            for index, rating in enumerate(average_ratings):
                self.average_ratings.append(rating)
#                 label_text = f"Average Online Rating for this Location: {rating}"
#                 new_label = tk.Label(self.all_tabs[index], text=label_text)
#                 new_label.pack()
#                 self.average_rating_labels.append(new_label)
        rating_frame = tk.Frame(self.all_tabs[4])
        rating_frame.pack()
        best = 0
        index = 0
        for rating in self.average_ratings:
            if float(rating) > best:
                best = float(rating)
                num = index
            index += 1
        if num == 0:
            place = 'U Club'
        elif num == 1:
            place = '20 Hawley'
        elif num == 2:
            place = 'Twin River Commons'
        elif num == 3:
            place = 'Printing House'
        text = tk.Label(rating_frame, text=f'\nThe highest rated place on average is {place} with an average rating of {best}.')
        text.config(font=('Sans Serif', 25, 'bold'))
        text.pack()

    def compare_commute(self):
        commute_frame = tk.Frame(self.all_tabs[4])
        commute_frame.pack()
        
        u_commute = self.commute_times[0]['Time to Reach Campus (minutes)'].astype(float)
        hawley_commute = self.commute_times[1]['Time to Reach Campus (minutes)'].astype(float)
        twin_commute = self.commute_times[2]['Time to Reach Campus (minutes)'].astype(float)
        printing_commute = self.commute_times[3]['Time to Reach Campus (minutes)'].astype(float)
        
        u_avg = u_commute.mean()
        hawley_avg = hawley_commute.mean()
        twin_avg = twin_commute.mean()
        printing_avg = printing_commute.mean()
        self.average_commute = [u_avg, hawley_avg, twin_avg, printing_avg]
        
        best = min(self.average_commute)
        
        if best == u_avg:
            place = 'U Club'
        elif best == hawley_avg:
            place = '20 Hawley'
        elif best == twin_avg:
            place = 'Twin River Commons'
        elif best == printing_avg:
            place = 'Printing House'
        
        # displays the lowest average commute time
        text = tk.Label(commute_frame, text=f'\nThe lowest commute time on average is from {place}, with {best} minutes to campus.')
        text.config(font=('Sans Serif', 25, 'bold'))
        text.pack()
    
    def import_averages(self):
        self.averages = [self.average_prices, self.average_ratings, self.average_commute]
        for index, average in enumerate(self.averages):
            num = 0
            for avg in average:
                if index == 0:
                    label_text = f'Average Rent for this Location: {avg}'
                elif index == 1:
                    label_text = f'Average Online Rating for this Location: {avg}'
                elif index == 2:
                    label_text = f'Average Commute for this Location: {avg}'
                new_label = tk.Label(self.all_tabs[num], text=label_text)
                new_label.pack()
                num += 1
                          
    def mainloop(self):
        self.root.mainloop()
        
window = MainWindow()
window.mainloop()